In [124]:
def authenticating(credential):
    logging('\n\nfunction>>>>>authenticating')
    
    auth = tweepy.OAuthHandler(credential["api_key"], credential["api_secret"])
    auth.set_access_token(credential["access_token"], credential["access_token_secret"])
    
    api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    return api

In [125]:
def print_and_retweet_tweet(api,tweet,dict_tweets_info):
    logging('\n\nfunction>>>>>print_and_retweet_tweet')
    
    try: 
        logging('appending infos to dictionary')
        dict_tweets_info['created_at'].append(str(tweet.created_at))
        dict_tweets_info['tweet_ID'].append(str(tweet.id))
        dict_tweets_info['user'].append(str(tweet.user.screen_name))
        dict_tweets_info['tweet_content'].append((tweet.text))
        dict_tweets_info['place'].append(str(tweet.place))
        dict_tweets_info['language'].append(str(tweet.lang))
        dict_tweets_info['source'].append(str(tweet.source_url).replace("http://twitter.com/download/",""))
    
        logging('----------------------------------------')
        logging('collected informations')
        logging('----------------------------------------')
 
        logging('dict_tweets_info: \n '+str(dict_tweets_info))
        logging('----------------------------------------\n\n\n\n')
        
        print(str(type(dict_tweets_info['tweet_content']))) 
        
    
        logging('retweeting ←←←←←←')
        api.retweet(tweet.id)
        logging('→→→→→→→ retweeted')
        return dict_tweets_info
    
    except tweepy.TweepError as e: 
        if e.api_code == 327:
            logging('You have already retweeted this Tweet')
            return False
        
    except tweepy.RateLimitError as e:
        logging('Excedeu limite por tempo?')
        logging('erro: '+str(e))
        logging('DORMINDO POR QUINZE MINUTOS')
        time.sleep(60 * 15)  # we supposedly saw a rate limit that is ignored after 15 min ??? so we should wait 15 min to retry 
        return False

In [126]:
def write_json_and_updates_value(path,incrementa_contagem_de_falha=False,inicializar = False):
    logging('\n\nfunction>>>>>write_json_and_updates_value')
    
    now = datetime.now()
    current_date = now.strftime("%d/%m/%Y")

    # try to read from file
    try:
        with open(path) as json_file:
            tweets_status = json.load(json_file)

    except Exception as e:
        print(str(e))

    # write on file
    # if our current date is the same, increase amount of tweets.
    # if our current date is different, amount is ZERO !!!!!!!!!!!!!!!!!!!!

    if inicializar or tweets_status['current_date'] != current_date: 
        logging('different dates, OR initializing, so we need to change the current_date value and also turn into 0 all the values')
        with open(path, 'w') as f:
            try:
                content = {"current_date": current_date,
                           "amount_of_tweets": 0,
                           "total_amount_including_failure":0}
                json.dump(content, f)

            except json.JSONDecodeError:
                logging('decode error but will try raw writing')
                f.write(contenting)
    else: 
        logging('same date!! so, just change the value of tweetts')
        if not incrementa_contagem_de_falha:
                logging('increases both keys , the including failure and the sucessed amounts')
                #vai incrementtar o total com falhas tb + o total dos sucessos
                tweets_status["amount_of_tweets"] = tweets_status["amount_of_tweets"]+1 
                tweets_status['total_amount_including_failure'] = tweets_status['total_amount_including_failure']+1
                with open(path, 'w') as f:
                    try:
                        json.dump(tweets_status, f)
                    except json.JSONDecodeError:
                        logging('decode error but will try raw writing')
                        f.write(contenting)
                    
        elif incrementa_contagem_de_falha:
                # vai incrementar SOMENTE chave com total de tweets, independente de ter falhado ou nao
                logging('INCREMENTANDO CHAVE DE CONTAGEM TOTAL DE TWEETS')
                     # increasing amount of the ones who failure 
                tweets_status['total_amount_including_failure'] = tweets_status['total_amount_including_failure']+1

                with open(path, 'w') as f:
                    try:
                        json.dump(tweets_status, f)

                    except json.JSONDecodeError:
                        logging('decode error but will try raw writing')
                        f.write(contenting)

In [127]:
def export_infos_to_csv(valid_tweet):
    logging('\n\nfunction>>>>>exporting_infos_to_csv')
    
    now = datetime.now()
    current_directory = os.getcwd()  
    timestamp = now.strftime("%d/%m/%Y").replace("/","-").replace(':',"-").replace(',','--').replace(" ","")

    CSV_path = current_directory+'\\arquivos_bot\\dados_exportados\\dados_'+timestamp+'.csv'
    logging('log path: '+str(CSV_path))
    
    logging('dict_tweets_info '+str(valid_tweet))
    
    df = pd.DataFrame.from_dict(valid_tweet) # turning into data frame
    df.to_csv(CSV_path,index=False)

In [128]:
def logging(text_to_log=""):
    
    # converte parâmetro que ele quer escrever no log em string, caso tenha sido enviado em outro formato
    text_to_log = str(text_to_log)
    
    # prepara data e timestamp pra dar append no arquivo de log (ou escrever um novo) + timestamp no conteúdo do arquivo
    now = datetime.now()
    date = now.strftime("%d/%m/%Y").replace("/","-")
    timestamp = now.strftime("%d/%m/%Y, %H:%M:%S")
    
    # busca diretório onde o robô está rodando e formata o path pro log do dia corrente
    current_directory = os.getcwd()  
    log_path = current_directory+'\\arquivos_bot\\logs\\log_'+date+'.txt'

    # se arquivo do dia já existir, só escreve nele o timestamp + conteúdo do parámetro
    # caso contrário, cria o arquivo de log daquele dia 
    with open(log_path, 'a+',encoding="utf-8") as log_file:
        log_file.write(timestamp+ ' - ' + text_to_log+'\n')
    
    # além de escrever no arquivo, printa no jupyter
    print(timestamp+ ' - ' + text_to_log)

In [129]:
def translate_special_text_to_ascii(original_text):
    translated_text = ''

    for character in original_text:
        if ord(character) >= 128:
            translated_text = translated_text + '"Chr(' + str(ord(character)) + ')"'
        else:
            translated_text = translated_text + character

    return translated_text

In [130]:
def main():
    
    # cria pastas que o robô usa caso não existam 
    current_directory = os.getcwd()
    if not os.path.exists(current_directory+'\\arquivos_bot\\logs'):
        pymsgbox.alert(text='Criando pasta de logs', title='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\logs')
        logging("Preparando ambiente")
        logging("Criando pasta de logs")
    
    if not os.path.exists(current_directory+'\\arquivos_bot\\controle'):
        pymsgbox.alert(text='Criando pasta de controle', tistle='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\controle')
        logging("Criando pasta de controle")

    if not os.path.exists(current_directory+'\\arquivos_bot\\dados_exportados'):
        pymsgbox.alert(text='Criando pasta de dados exportados', title='Preparando bot', button='OK',timeout=4500)
        os.makedirs(current_directory+'\\arquivos_bot\\dados_exportados')
        logging("Criando pasta de dados exportados")
    
    pymsgbox.alert('Pastas necessárias para o robô conferidas, iniciando o bot', 'Starting bot',timeout=5000)
    logging(" --------- INICIANDO ROBÔ ---------")
    
    # checking if control json exists, otherwise we create it
    if not os.path.exists(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json'):
        logging("arquivo json não encontrado")
        now = datetime.now()
        date = now.strftime("%d/%m/%Y")
        write_json_and_updates_value(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json',incrementa_contagem_de_falha=False,inicializar = True)
    
    credential =  {
                    "api_key" : credentials.API_KEY,
                    "api_secret" : credentials.API_SECRET,
                    "bearer_token" : credentials.BEARER_TOKEN,
                    "access_token" : credentials.ACCESS_TOKEN,
                    "access_token_secret" : credentials.ACCESS_TOKEN_SECRET
                    }
    
    api = authenticating(credential)
    
    dict_tweets_info = {
        "created_at": [],
        "tweet_ID": [],
        "user": [],
        "tweet_content": [],
        "place": [],
        "language": [],
        "source": [] 
    }
    
    for tweet in tweepy.Cursor(api.search, q='zolpidem').items(1100):
        
        with open(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json') as json_file:
            tweets_status = json.load(json_file)
            if tweets_status["amount_of_tweets"] == 999 and tweets_status == date:
                sys.exit('DAILY LIMIT REACHED, CANT RETWEET MORE THAN 1000 TWEETS')
                
        valid_tweet = print_and_retweet_tweet(api,tweet,dict_tweets_info)
        
        if not valid_tweet:
            logging('Tweet is not valid for some reason thus we retrieve another one')
            write_json_and_updates_value(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json',incrementa_contagem_de_falha=True)
            continue
            
        if isinstance(valid_tweet,dict):
            logging('Ok, we received a dict as return, we may export the results now')
            export_infos_to_csv(valid_tweet)
            write_json_and_updates_value(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json',incrementa_contagem_de_falha=False)
        else:
            logging('Unexpected return for print_and_retweet_tweet different than dict or false!! content: '+str(valid_tweet))
            write_json_and_updates_value(current_directory+'\\arquivos_bot\\controle\\amount_of_tweets_from_today.json',incrementa_contagem_de_falha=False)
        logging("Waiting 2 min for retrieve another tweet cuz we like safety")
        time.sleep(60*2) # sleep 2 min, so we dont reach the limit 100 tweets per hour
        
    logging('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ FIM DA LAP $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ ')   
    pymsgbox.alert('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ \n         FIM DA LAP\n $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$', 'End of times ',timeout=40000)

In [ ]:
import import_ipynb
import credentials
import tweepy
import time
from datetime import date, datetime 
import os
import pymsgbox 
import pandas as pd
import json
import sys

main()

03/01/2021, 00:11:09 -  --------- INICIANDO ROBÔ ---------
03/01/2021, 00:11:09 - 

function>>>>>authenticating
03/01/2021, 00:11:10 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 00:11:10 - appending infos to dictionary
03/01/2021, 00:11:10 - ----------------------------------------
03/01/2021, 00:11:10 - collected informations
03/01/2021, 00:11:10 - ----------------------------------------
03/01/2021, 00:11:10 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21'], 'tweet_ID': ['1345566389369757697'], 'user': ['tnyssx'], 'tweet_content': ['tomar um comprimido de zolpidem de um chazin de camomila pra dormir por 5 dias seguidos'], 'place': ['None'], 'language': ['pt'], 'source': ['https://mobile.twitter.com']}
03/01/2021, 00:11:10 - ----------------------------------------




<class 'list'>
03/01/2021, 00:11:10 - retweeting ←←←←←←
03/01/2021, 00:11:10 - →→→→→→→ retweeted
03/01/2021, 00:11:10 - Ok, we received a dict as return, we may export the results now
03/01/2021, 00:1

03/01/2021, 00:19:12 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 00:19:12 - appending infos to dictionary
03/01/2021, 00:19:12 - ----------------------------------------
03/01/2021, 00:19:12 - collected informations
03/01/2021, 00:19:12 - ----------------------------------------
03/01/2021, 00:19:12 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53'], 'tweet_ID': ['1345566389369757697', '1345566249279913986', '1345566043700387840', '1345565179757604864', '1345565016305631233'], 'user': ['tnyssx', 'bokoita_touhou', 'nicogsworld', 'fckpieck', 'opia36'], 'tweet_content': ['tomar um comprimido de zolpidem de um chazin de camomila pra dormir por 5 dias seguidos', '@Zolpidem_3mg こないだ大阪でｳﾏﾁｬﾝの擬人化見つけたけど主に体型のデフォルメがむっちゃ入ってたな', '@JusteTara Zolpidem', 'eu realmente m gosto de tomar remedio de dormir, tipo rivotril etc, mas amo o zolpidem, obv tive experiências ruins… https://t.co/qmeBrWcBq

03/01/2021, 00:23:14 - →→→→→→→ retweeted
03/01/2021, 00:23:14 - Ok, we received a dict as return, we may export the results now
03/01/2021, 00:23:14 - 

function>>>>>exporting_infos_to_csv
03/01/2021, 00:23:14 - log path: C:\Users\gabri\Documents\Retweet-Bot\arquivos_bot\dados_exportados\dados_03-01-2021.csv
03/01/2021, 00:23:14 - dict_tweets_info {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42'], 'tweet_ID': ['1345566389369757697', '1345566249279913986', '1345566043700387840', '1345565179757604864', '1345565016305631233', '1345565008491671553', '1345564967374704640'], 'user': ['tnyssx', 'bokoita_touhou', 'nicogsworld', 'fckpieck', 'opia36', 'AlannaSerafim', 'wjdeardorff'], 'tweet_content': ['tomar um comprimido de zolpidem de um chazin de camomila pra dormir por 5 dias seguidos', '@Zolpidem_3mg こないだ大阪でｳﾏﾁｬﾝの擬人化見つけたけど主に体型のデフォルメがむっちゃ入ってたな', '@JusteTara Zolpidem'

03/01/2021, 00:27:15 - Tweet is not valid for some reason thus we retrieve another one
03/01/2021, 00:27:15 - 

function>>>>>write_json_and_updates_value
03/01/2021, 00:27:15 - same date!! so, just change the value of tweetts
03/01/2021, 00:27:15 - INCREMENTANDO CHAVE DE CONTAGEM TOTAL DE TWEETS
03/01/2021, 00:27:15 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 00:27:15 - appending infos to dictionary
03/01/2021, 00:27:15 - ----------------------------------------
03/01/2021, 00:27:15 - collected informations
03/01/2021, 00:27:15 - ----------------------------------------
03/01/2021, 00:27:15 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28'], 'tweet_ID': ['1345566389369757697', '1345566249279913986', '1345566043700387840', '1345565179757604864', '134556501630563123

03/01/2021, 00:31:16 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 00:31:16 - appending infos to dictionary
03/01/2021, 00:31:16 - ----------------------------------------
03/01/2021, 00:31:16 - collected informations
03/01/2021, 00:31:16 - ----------------------------------------
03/01/2021, 00:31:16 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13'], 'tweet_ID': ['1345566389369757697', '1345566249279913986', '1345566043700387840', '1345565179757604864', '1345565016305631233', '1345565008491671553', '1345564967374704640', '1345564714860998659', '1345562134990602241', '1345561639639162880', '1345561449490472961', '1345559562104025088'], 'user': ['tnyssx', 'bokoita_touhou', 'nicogsworld', 'fckpieck', 'opia36', 'AlannaSera

...

03/01/2021, 02:39:59 - →→→→→→→ retweeted
03/01/2021, 02:39:59 - Ok, we received a dict as return, we may export the results now
03/01/2021, 02:39:59 - 

function>>>>>exporting_infos_to_csv
03/01/2021, 02:39:59 - log path: C:\Users\gabri\Documents\Retweet-Bot\arquivos_bot\dados_exportados\dados_03-01-2021.csv
03/01/2021, 02:39:59 - dict_tweets_info {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13', '2021-01-03 02:36:09', '2021-01-03 02:34:22', '2021-01-03 02:32:32', '2021-01-03 02:30:57', '2021-01-03 02:18:07', '2021-01-03 02:12:31', '2021-01-03 02:02:31', '2021-01-03 01:58:24', '2021-01-03 01:56:46', '2021-01-03 01:55:24', '2021-01-03 01:54:43', '2021-01-03 01:46:51', '2021-01-03 01:33:49', '2021-01-03 01:27:57', '2021-01-03 01:26:21', '2021-01-03 0

03/01/2021, 02:41:59 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 02:41:59 - appending infos to dictionary
03/01/2021, 02:41:59 - ----------------------------------------
03/01/2021, 02:41:59 - collected informations
03/01/2021, 02:41:59 - ----------------------------------------
03/01/2021, 02:41:59 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13', '2021-01-03 02:36:09', '2021-01-03 02:34:22', '2021-01-03 02:32:32', '2021-01-03 02:30:57', '2021-01-03 02:18:07', '2021-01-03 02:12:31', '2021-01-03 02:02:31', '2021-01-03 01:58:24', '2021-01-03 01:56:46', '2021-01-03 01:55:24', '2021-01-03 01:54:43', '2021-01-03 01:46:51', '2021-01-03 01:33:49', '2021-01-03 01:27:57', '2021-01-03 01:26:21', '2021-01-03 01:19:28', '2021-01

03/01/2021, 02:41:59 - →→→→→→→ retweeted
03/01/2021, 02:41:59 - Ok, we received a dict as return, we may export the results now
03/01/2021, 02:41:59 - 

function>>>>>exporting_infos_to_csv
03/01/2021, 02:41:59 - log path: C:\Users\gabri\Documents\Retweet-Bot\arquivos_bot\dados_exportados\dados_03-01-2021.csv
03/01/2021, 02:41:59 - dict_tweets_info {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13', '2021-01-03 02:36:09', '2021-01-03 02:34:22', '2021-01-03 02:32:32', '2021-01-03 02:30:57', '2021-01-03 02:18:07', '2021-01-03 02:12:31', '2021-01-03 02:02:31', '2021-01-03 01:58:24', '2021-01-03 01:56:46', '2021-01-03 01:55:24', '2021-01-03 01:54:43', '2021-01-03 01:46:51', '2021-01-03 01:33:49', '2021-01-03 01:27:57', '2021-01-03 01:26:21', '2021-01-03 0

03/01/2021, 02:43:59 - 

function>>>>>print_and_retweet_tweet
03/01/2021, 02:43:59 - appending infos to dictionary
03/01/2021, 02:43:59 - ----------------------------------------
03/01/2021, 02:43:59 - collected informations
03/01/2021, 02:43:59 - ----------------------------------------
03/01/2021, 02:43:59 - dict_tweets_info: 
 {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13', '2021-01-03 02:36:09', '2021-01-03 02:34:22', '2021-01-03 02:32:32', '2021-01-03 02:30:57', '2021-01-03 02:18:07', '2021-01-03 02:12:31', '2021-01-03 02:02:31', '2021-01-03 01:58:24', '2021-01-03 01:56:46', '2021-01-03 01:55:24', '2021-01-03 01:54:43', '2021-01-03 01:46:51', '2021-01-03 01:33:49', '2021-01-03 01:27:57', '2021-01-03 01:26:21', '2021-01-03 01:19:28', '2021-01

03/01/2021, 02:44:00 - →→→→→→→ retweeted
03/01/2021, 02:44:00 - Ok, we received a dict as return, we may export the results now
03/01/2021, 02:44:00 - 

function>>>>>exporting_infos_to_csv
03/01/2021, 02:44:00 - log path: C:\Users\gabri\Documents\Retweet-Bot\arquivos_bot\dados_exportados\dados_03-01-2021.csv
03/01/2021, 02:44:00 - dict_tweets_info {'created_at': ['2021-01-03 03:03:21', '2021-01-03 03:02:47', '2021-01-03 03:01:58', '2021-01-03 02:58:32', '2021-01-03 02:57:53', '2021-01-03 02:57:52', '2021-01-03 02:57:42', '2021-01-03 02:56:42', '2021-01-03 02:46:26', '2021-01-03 02:44:28', '2021-01-03 02:43:43', '2021-01-03 02:36:13', '2021-01-03 02:36:09', '2021-01-03 02:34:22', '2021-01-03 02:32:32', '2021-01-03 02:30:57', '2021-01-03 02:18:07', '2021-01-03 02:12:31', '2021-01-03 02:02:31', '2021-01-03 01:58:24', '2021-01-03 01:56:46', '2021-01-03 01:55:24', '2021-01-03 01:54:43', '2021-01-03 01:46:51', '2021-01-03 01:33:49', '2021-01-03 01:27:57', '2021-01-03 01:26:21', '2021-01-03 0